# Part 3 – Multi-Agent Mayhem: The Router Agent 🚏

In this section, we introduce the **Router Agent**, a top-level controller whose only job is to analyze a user query and decide **which specialized agent** should handle it.

---

## Concept: Why Router Agents? 🤔
- A single agent has limited abilities.  
- Complex user queries often span multiple domains (travel, food, transportation, events).  
- The Router Agent acts as a **dispatcher**:
  - It does **not** answer the query itself.  
  - It **returns the name** of the best specialized agent for the job.  

---




Part 0: Setup & Authentication 🔑

In [2]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [3]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

✅ API Key configured successfully! Let the fun begin.


In [4]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"


## Step 1 – Define Specialized Agents 🧑‍💻

In [ ]:
#-- Agent Definitions for our Specialist Team --

day_trip_agent = Agent(
    name="day_trip_agent",
    model="gemini-2.5-flash",
    description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
    instruction="""
    You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

    Your Mission:
    Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

    Guidelines:
    1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
    2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
    3. **Real-Time Focus**: Search for current operating hours and special events.
    4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

    RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
    """,
    tools=[google_search]

)

foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an expert food critical. Your goal is to find the absolute best food, restaurant, or culinary experiences basedAgent" \
    "on a user's request. When you recommend a place, state its name clearly. For example: 'The best sushi is at **Jin Sho**. "
)

weekend_guide_agent=Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

transportation_agent=Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an navigatoin assistant. Given a starting point and destination, provide clear directions on how to get from the start to the end."
)


# Step 2 – Define the Router Agent 🚏

In [6]:
# --- The Brain of the Operation: The Router Agent ---
# We update the router's instructions to know about the new 'combo' task.

router_agent=Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
        You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_combo': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We'll create a dictionary of all our individual worker agents
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "transportation_agent": transportation_agent, # Add the new agent!
}

print("🤖 Agent team assembled for sequential workflows!")

🤖 Agent team assembled for sequential workflows!


# Step 3 – Sequential Workflow (Router → Specialist) 🔀

In [7]:
# --- Let's Test the Sequential Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the COMBO
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        #1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent,query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route == 'find_and_navigate_combo':
            print("\n--- Starting Find and Navigate Combo Workflow ---")

            # STEP 2a: Run the foodie_agent first
            foodie_session = await session_service.create_session(app_name=foodie_agent.name, user_id=my_user_id)
            foodie_response = await run_agent_query(foodie_agent, query, foodie_session, my_user_id)

            # Step 2b: Extract the destination from the first agent's response
            #(This is a simple regex, a more robust solution might use a structured output format)
            match = re.search(r'\*\*(.*?)\*\*', foodie_response)
            if not match:
                print("🚨 Could not determine the restaurant name from the response.")
                continue
            destination = match.group(1)
            print(f" Extract Destination: {destination}")

            #Step 2c: Create a new query and run the transportation_agent
            directions_query = f"Give me directions to {destination} from the Palo Alto Caltrain station."
            print(f"\n🗣️ New Query for Transport Agent: '{directions_query}'")
            transport_session = await session_service.create_session(app_name=transportation_agent.name, user_id=my_user_id)
            await run_agent_query(transportation_agent, directions_query, transport_session, my_user_id)

            print("--- Combo Workflow Complete ---")

        elif chosen_route in worker_agents:
            #This is a simple, single-agent route
            worker_agent = worker_agents[chosen_route]
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session,my_user_id)
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")    

await run_sequential_app()




🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '7254c29c-d1c3-4e1d-9166-7daf266df77a'...
🚦 Router has selected route: 'foodie_agent'

🚀 Running query for agent: 'foodie_agent' in session: '3a527d20-6c37-4934-8706-a1fd9c31d1e6'...
EVENT: content=Content(
  parts=[
    Part(
      text="""If you're seeking the absolute best sushi experience in Palo Alto, look no further than **Jin Sho**.

Jin Sho is consistently lauded for its exquisite Japanese cuisine and the meticulous care taken in preparing its unique fish selection. Reviewers frequently highlight the exceptional freshness of their sushi, with specific praise for the "buttery and soft" otoro and "super creamy and sweet" uni, noting how the fish melts in your mouth. Beyond its fresh fish, Jin Sho is also recognized for its yellowtail jalapeno sashimi and spicy tuna rolls, and even holds the distinction of bei

If you're seeking the absolute best sushi experience in Palo Alto, look no further than **Jin Sho**.

Jin Sho is consistently lauded for its exquisite Japanese cuisine and the meticulous care taken in preparing its unique fish selection. Reviewers frequently highlight the exceptional freshness of their sushi, with specific praise for the "buttery and soft" otoro and "super creamy and sweet" uni, noting how the fish melts in your mouth. Beyond its fresh fish, Jin Sho is also recognized for its yellowtail jalapeno sashimi and spicy tuna rolls, and even holds the distinction of being a favorite of Steve Jobs. While it is considered high-end and can be pricey, the quality of the food makes it a top choice for a premier sushi experience.

--------------------------------------------------


🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'd6e3ed36-5013-4cae-85b6-f2f766bbbef8'...
🚦 Router has selected route: 'weekend_guide_agent'

🚀 Running query for agent: 'weekend_guide_agent' in session: 'a9c9643f-cd24-45e3-b4db-65f0c372b2b5'...
EVENT: content=Content(
  parts=[
    Part(
      text="""This weekend, August 22-24, 2025, offers a variety of outdoor concerts and music festivals across different locations:

**United States:**

*   **Connecticut:**
    *   The **Black Bear Music Fest** is taking place at the Harwinton Fairgrounds from August 22-24. This seventh annual festival features over 50 bands across four stages, playing rock, blues, folk, and bluegrass. It also offers on-site camping, food trucks, and family-friendly activities.
*   **New York City, New York:**
    *   The **Charlie Par

This weekend, August 22-24, 2025, offers a variety of outdoor concerts and music festivals across different locations:

**United States:**

*   **Connecticut:**
    *   The **Black Bear Music Fest** is taking place at the Harwinton Fairgrounds from August 22-24. This seventh annual festival features over 50 bands across four stages, playing rock, blues, folk, and bluegrass. It also offers on-site camping, food trucks, and family-friendly activities.
*   **New York City, New York:**
    *   The **Charlie Parker Jazz Festival** kicks off this weekend. On Friday, August 22, Ron Carter's Great Big Band and April May Webb will perform at Marcus Garvey Park in Manhattan. The festival continues on Saturday, August 23, with the Branford Marsalis Quartet, Nicholas Payton, Bertha Hope Quintet, Georgia Heers, and DJ KulturedChild aka Angelika Beener, also at Marcus Garvey Park.
    *   **Bryant Park Picnic Performances** in Manhattan will host "Belongó: Arturo O'Farrill and the Afro-Latin Jazz Orchestra" on Friday, August 22. This is a free outdoor festival that also offers free blanket loans and food/drink for purchase.
*   **Des Moines, Iowa:**
    *   The **World Food & Music Festival** runs from Friday, August 22, to Sunday, August 24, at Western Gateway Park. This dynamic three-day event features international cuisine, live performances, cultural activities, and global beverages.
*   **Highland Park, Illinois:**
    *   The **Ravinia Festival**, North America's longest-running outdoor music festival, has several events this weekend. On Friday, August 22, Maren Morris will perform as part of "The Dreamsicle Tour."
*   **Isle, Minnesota:**
    *   The **Rural Music Fest** is scheduled for August 22-24. This free Christian music festival will also include food trucks and a kids' zone. Donations are appreciated.

**International:**

*   **Portsmouth, UK:**
    *   **Victorious Festival** will be held from August 22-24. Described as one of the most picturesque festivals in the UK, it features world-class music, comedy, and is known for being family-friendly.
*   **Vancouver, Canada:**
    *   The **PNE Fair's Summer Night Concerts** include several performances this weekend. While the concerts are held in the Pacific Coliseum due to ongoing construction, admission to the Fair is included with a concert ticket, offering an overall outdoor festival experience. Flo Rida performs on August 22, Leon Bridges on August 23, and Marianas Trench on August 24.

--------------------------------------------------


🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'f98d1356-8694-4007-b27b-c133f6ea2d25'...
🚦 Router has selected route: 'find_and_navigate_combo'

--- Starting Find and Navigate Combo Workflow ---

🚀 Running query for agent: 'foodie_agent' in session: 'e25ad188-f0fa-474e-94ee-76e9bea79561'...
EVENT: content=Content(
  parts=[
    Part(
      text="""The best sushi in Palo Alto is widely considered to be at **Jin Sho**. It's frequently praised for its incredibly fresh fish, with reviewers highlighting the buttery otoro and creamy uni. The restaurant is also known for its expertly crafted dishes, including their renowned black cod with miso, and offers a chic setting with modern pastel decor. It was even noted as a favorite of Steve Jobs.

To get to Jin Sho fro

The best sushi in Palo Alto is widely considered to be at **Jin Sho**. It's frequently praised for its incredibly fresh fish, with reviewers highlighting the buttery otoro and creamy uni. The restaurant is also known for its expertly crafted dishes, including their renowned black cod with miso, and offers a chic setting with modern pastel decor. It was even noted as a favorite of Steve Jobs.

To get to Jin Sho from the Palo Alto Caltrain station:

1.  **Walk southwest on University Avenue** from the Caltrain station platform.
2.  **Turn left onto California Avenue.**
3.  **Jin Sho** is located at 825 Emerson St, Palo Alto, CA 94301, which is very close to California Avenue. You will find it on your left.

The walk from the Caltrain station to Jin Sho is approximately 0.7 miles and takes about 10-15 minutes.

--------------------------------------------------

 Extract Destination: Jin Sho

🗣️ New Query for Transport Agent: 'Give me directions to Jin Sho from the Palo Alto Caltrain station.'

🚀 Running query for agent: 'transportation_agent' in session: '48a051ef-d9e9-4630-a698-e32e2a972a47'...
EVENT: content=Content(
  parts=[
    Part(
      text="""To get to Jin Sho from the Palo Alto Caltrain station, you can expect a walk of approximately 15-20 minutes, or a short drive. Jin Sho is located at 454 S California Ave, Palo Alto, CA 94306.

Here are the walking directions:

1.  **Exit the Palo Alto Caltrain Station** (located at 95 University Ave, Palo Alto, CA 94301) and head southeast on University Avenue.
2.  Continue on University Avenue for about 0.3 miles.
3.  Turn right onto Alma Street and walk for a short block.
4.  Turn left onto California Avenue.
5.  Walk down California Avenue for approximately 0.5 miles, and Jin Sho will be on your right at 454 S California Ave."""
    ),
  ],

To get to Jin Sho from the Palo Alto Caltrain station, you can expect a walk of approximately 15-20 minutes, or a short drive. Jin Sho is located at 454 S California Ave, Palo Alto, CA 94306.

Here are the walking directions:

1.  **Exit the Palo Alto Caltrain Station** (located at 95 University Ave, Palo Alto, CA 94301) and head southeast on University Avenue.
2.  Continue on University Avenue for about 0.3 miles.
3.  Turn right onto Alma Street and walk for a short block.
4.  Turn left onto California Avenue.
5.  Walk down California Avenue for approximately 0.5 miles, and Jin Sho will be on your right at 454 S California Ave.

--------------------------------------------------

--- Combo Workflow Complete ---


# Part 2 (The ADK Way): Multi-Agent Mayhem with SequentialAgent 🧠→⛓️→🤖